## Comparing networkx and graph-tool

The most popular graph analysis libraries for Python are `networkx` and `graph-tool`. While `networkx` offers a very accessible Python API and a large variety of out-of-the-box graph algorithms, `graph-tool` is compiled down into C and claims to offer substantial performance gains. I want to validate those claims here.

In [1]:
import time

import numpy as np
import networkx as nx
import graph_tool.all as gt

import cc_graph_ops

In [2]:
n = 5000
edges = [(0, 1)]
for i in range(2, n):
    edges.append((i, np.random.randint(i)))

nxG = nx.Graph()
nxG.add_nodes_from(range(n))
nxG.add_edges_from(edges)

gtG = gt.Graph(directed=False)
gtG.add_vertex(n)
gtG.add_edge_list(edges)

In [3]:
def time_method(name, func, arg, template="{:<20}{:>10.3f} s"):
    start = time.time()
    func(arg)
    end = time.time()
    print(template.format(name, end - start))

In [4]:
print("Evaluating graph-tool PageRank")
time_method("gt PageRank", gt.pagerank, gtG)
print()
print("Evaluating networkx PageRank Algorithms")
time_method("nx PageRank", nx.pagerank, nxG)
time_method("nx PageRank (Numpy)", nx.pagerank_numpy, nxG)
time_method("nx PageRank (Scipy)", nx.pagerank_scipy, nxG)

Evaluating graph-tool PageRank
gt PageRank              0.005 s

Evaluating networkx PageRank Algorithms
nx PageRank              0.574 s
nx PageRank (Numpy)     55.845 s
nx PageRank (Scipy)      0.027 s


`graph-tool` definitely takes the cake on this one. To be honest, I'm unsure why the `nx.pagerank_numpy` algorithm turns out to run the slowest. Maybe it's not optimized for sparse graphs?

In [5]:
print("Comparing Vertex Betweeness Centrality")
time_method("gt Betweenness Centrality", gt.betweenness, gtG)
time_method("nx Betweeeness Centrality", nx.betweenness_centrality, nxG)

Comparing Vertex Betweeness Centrality
gt Betweenness Centrality     0.396 s
nx Betweeeness Centrality    65.690 s


In [6]:
print("Comparing K-Cores")
time_method("gt kcores", gt.kcore_decomposition, gtG)
# Note: the nx.k_core actually returns the subgraph for a specified k; we want to get for all k's for a fair comparison
time_method("nx kcores", nx.core_number, nxG)

Comparing K-Cores
gt kcores                0.000 s
nx kcores                0.016 s


We see from the above tests that (at least for sparse, small graphs that can fit in RAM), `graph-tool` consistenly out-performs `networkx`. My recommendation is that we primarily use `graph-tool` once we start performing these computations on our dataset and reserve `networkx` for exploratory prototyping and for algorithms not present in the `graph-tool` library.

## Benchmarking PageRank

In [2]:
def generate_edge_list(v, ev_ratio=20):
    """Creates an edge lists to construct a graph-tool graph.
    
    Parameters:
    v: number of vertices
    ev_ratio: ratio of edges to vertices, equals 0.5 * average degree
    
    Returns:
    list of edges (v1, v2)
    """
    edges = [(0, 1)]
    for i in range(2, v):
        for _ in range(ev_ratio):
            edges.append((i, np.random.randint(i)))
    return edges

for v in [1e2, 5e2, 1e3, 5e3, 1e4, 5e4, 1e5, 5e5]:
    v = int(v)
    tic = time.time()
    gtG = gt.Graph(directed=True)
    gtG.add_vertex(v)
    gtG.add_edge_list(generate_edge_list(v))
    toc = time.time()
    print(f'constructing graph on {v} vertices took {(toc - tic):.3f} s')
    
    tic = time.time()
    _ = gt.pagerank(gtG)
    toc = time.time()
    print(f'PageRank took {(toc-tic):.3f} s')

constructing graph on 100 vertices took 0.016 s
PageRank took 0.001 s
constructing graph on 500 vertices took 0.044 s
PageRank took 0.001 s
constructing graph on 1000 vertices took 0.098 s
PageRank took 0.001 s
constructing graph on 5000 vertices took 0.398 s
PageRank took 0.005 s
constructing graph on 10000 vertices took 0.684 s
PageRank took 0.013 s
constructing graph on 50000 vertices took 3.214 s
PageRank took 0.119 s
constructing graph on 100000 vertices took 6.319 s
PageRank took 0.265 s
constructing graph on 500000 vertices took 31.603 s
PageRank took 3.646 s


In [1]:
import time

import json
import numpy as np
import networkx as nx
import graph_tool.all as gt

import cc_graph_ops

In [2]:
with open('fdg_input_file.json', 'r') as f:
    data = json.load(f)
    
nodes = data['nodes']
links = data['links']

In [4]:
import resource

def memory_limit(limit):
    """prevents python from using more than LIMIT bytes"""
    soft, hard = resource.getrlimit(resource.RLIMIT_AS)
    resource.setrlimit(resource.RLIMIT_AS, (limit, hard))
    
memory_limit(4*1024*1024*1024)

In [5]:
# 30000 nodes is around the limit when we 
# restrict to 4GB of memory usage

limit = 30000

gtG = gt.Graph(directed=True)
gtG.vertex_properties['id'] = gtG.new_vertex_property('string')
vertices = dict()
for node in nodes[:limit]:
    v = gtG.add_vertex()
    vertices[node['id']] = v
    gtG.vertex_properties['id'][v] = node['id']
edges = []
for link in links:
    if link['source'] in vertices and link['target'] in vertices:
        src = vertices[link['source']]
        dest = vertices[link['target']]
        for i in range(link['value']):
            gtG.add_edge(src, dest)

## Using the GPU

However, to my knowledge, neither of these libraries are accessing the GPU. Below, I experiment with one of the more promising GPU graph analytics libraries, [`cuGraph`](https://github.com/rapidsai/cugraph) from [rapids.ai](https://rapids.ai/).

The following code is modified from a Rapids AI [tutorial](https://github.com/rapidsai/cugraph/blob/branch-0.13/notebooks/centrality/Katz.ipynb). **Requires NVIDIA Pascal or newer GPU architecture**.

In [7]:
import cugraph
import cudf
from collections import OrderedDict

/home/sohom/miniconda3/envs/linked_commons/lib/python3.8/site-packages/cudf/utils/gpu_utils.py:92: UserWarning: You will need a GPU with NVIDIA Pascal™ or newer architecture
Detected GPU 0: GeForce GTX 950M 
Detected Compute Capability: 5.0
  warnings.warn(


In [8]:
import networkx as nx

In [9]:
max_iter = 100
tol = 1e-5

datafile = '../karate-data.csv'

In [ ]:
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst')

In [ ]:
df_page = cugraph.pagerank(G)

for i in range(len(df_page)):
    print("vertex " + str(df_page['vertex'].iloc[i]) + " PageRank is " + str(df_page['pagerank'].iloc[i]))